In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gc  # Import garbage collector module

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence

def calculate_difference_image(image1, image2):
    return cv2.absdiff(image1, image2)

class ImageDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.images) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.images[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)

def load_images_and_difference(authentic_dir, tampered_dir):
    images = []
    labels = []

    print("Loading authentic images...")
    for filename in os.listdir(authentic_dir):
        if filename.lower().endswith(('.jpg', '.bmp')):
            filepath = os.path.join(authentic_dir, filename)
            img = cv2.imread(filepath, cv2.IMREAD_COLOR)
            if img is not None:
                img_resized = cv2.resize(img, (224, 224)).astype(np.float32) / 255.0
                images.append(img_resized)
                labels.append(0)

    print("Loading tampered images and calculating differences...")
    for filename in os.listdir(tampered_dir):
        if filename.lower().endswith(('.jpg', '.tif')):
            tampered_img = cv2.imread(os.path.join(tampered_dir, filename), cv2.IMREAD_COLOR)
            tampered_img_resized = cv2.resize(tampered_img, (224, 224)).astype(np.float32) / 255.0
            
            match_found = False
            parts = filename.split("_")
            if len(parts) >= 7:
                source_id = parts[4][:3] + "_" + parts[4][3:]
                target_id = parts[5][:3] + "_" + parts[5][3:]
                for id_ in [source_id, target_id]:
                    for ext in ['.jpg', '.bmp']:
                        auth_filename = f"Au_{id_}{ext}"
                        auth_filepath = os.path.join(authentic_dir, auth_filename)
                        if os.path.exists(auth_filepath):
                            auth_img = cv2.imread(auth_filepath, cv2.IMREAD_COLOR)
                            auth_img_resized = cv2.resize(auth_img, (224, 224)).astype(np.float32) / 255.0
                            difference_img = calculate_difference_image(tampered_img_resized, auth_img_resized)
                            images.append(difference_img)
                            labels.append(1)
                            match_found = True
                            break
            if not match_found:
                print(f"No authentic match found for {filename}, using tampered image directly.")
                images.append(tampered_img_resized)
                labels.append(1)

    return np.array(images), np.array(labels)

authentic_dir = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au'
tampered_dir = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp'

images, labels = load_images_and_difference(authentic_dir, tampered_dir)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Train set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

class TrainingMonitor(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch+1}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"Finished epoch {epoch+1}")
        print(f"Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, Val Loss: {logs['val_loss']}, Val Accuracy: {logs['val_accuracy']}")

batch_size = 16
train_generator = ImageDataGenerator(X_train, y_train, batch_size)
test_generator = ImageDataGenerator(X_test, y_test, batch_size)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[TrainingMonitor()],
    verbose=2)

final_loss, final_accuracy = model.evaluate(test_generator)
print(f"Final test loss: {final_loss}, Final test accuracy: {final_accuracy}")

# Save the entire model
model.save('MobileNetV2_BC.h5')
print("Saved model to disk.")


Loading authentic images...
Loading tampered images and calculating differences...
No authentic match found for Tp_S_CNN_S_N_cha0003_cha00003_00323.tif, using tampered image directly.
No authentic match found for Tp_S_NNN_M_N_pla0006_pla00006_01128.tif, using tampered image directly.
Train set size: 10091, Test set size: 2523




Starting epoch 1
Epoch 1/10



Finished epoch 1
Loss: 0.08299383521080017, Accuracy: 0.9739371538162231, Val Loss: 0.04672575369477272, Val Accuracy: 0.9849385619163513
631/631 - 183s - loss: 0.0830 - accuracy: 0.9739 - val_loss: 0.0467 - val_accuracy: 0.9849 - 183s/epoch - 291ms/step
Starting epoch 2
Epoch 2/10
Finished epoch 2
Loss: 0.04795760661363602, Accuracy: 0.9854325652122498, Val Loss: 0.04245280846953392, Val Accuracy: 0.9857312440872192
631/631 - 165s - loss: 0.0480 - accuracy: 0.9854 - val_loss: 0.0425 - val_accuracy: 0.9857 - 165s/epoch - 261ms/step
Starting epoch 3
Epoch 3/10
Finished epoch 3
Loss: 0.03111722320318222, Accuracy: 0.9908829927444458, Val Loss: 0.04738790541887283, Val Accuracy: 0.9877130389213562
631/631 - 168s - loss: 0.0311 - accuracy: 0.9909 - val_loss: 0.0474 - val_accuracy: 0.9877 - 168s/epoch - 266ms/step
Starting epoch 4
Epoch 4/10
Finished epoch 4
Loss: 0.024665286764502525, Accuracy: 0.9914775490760803, Val Loss: 0.048341892659664154, Val Accuracy: 0.9861276149749756
631/631 - 1

C:\Users\sathy\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved model to disk.


In [1]:
def prepare_single_image(test_image_path, reference_image_path):
    test_img = cv2.imread(test_image_path, cv2.IMREAD_COLOR)
    test_img_resized = cv2.resize(test_img, (224, 224)).astype(np.float32) / 255.0

    reference_img = cv2.imread(reference_image_path, cv2.IMREAD_COLOR)
    reference_img_resized = cv2.resize(reference_img, (224, 224)).astype(np.float32) / 255.0

    difference_img = calculate_difference_image(test_img_resized, reference_img_resized)
    difference_img = np.expand_dims(difference_img, axis=0)
    return difference_img

In [4]:
# Load the saved model
loaded_model = load_model('MobileNetV2_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_nat_00003.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NNN_S_N_nat00003_cha00096_00623.tif'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] > 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 1s 879ms/step
Prediction (0 for Authentic, 1 for Tampered): [1.]
The image is likely tampered.


In [6]:
# Load the saved model
loaded_model = load_model('MobileNetV2_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_sec_10109.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NRN_M_N_sec10105_sec10109_10333.tif'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] > 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 2s 2s/step
Prediction (0 for Authentic, 1 for Tampered): [0.93142855]
The image is likely tampered.


In [8]:
# Load the saved model
loaded_model = load_model('MobileNetV2_BC.h5')

test_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au\Au_pla_00050.jpg'
reference_image_path = 'D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Tp\Tp_D_NRN_S_N_ind00009_pla00050_10423.tif'

difference_image = prepare_single_image(test_image_path, reference_image_path)
prediction = loaded_model.predict(difference_image)

print(f"Prediction (0 for Authentic, 1 for Tampered): {prediction[0]}")
if prediction[0] > 0.5:
    print("The image is likely tampered.")
else:
    print("The image is likely authentic.")

1/1 [==============================] - 1s 728ms/step
Prediction (0 for Authentic, 1 for Tampered): [0.99999994]
The image is likely tampered.


In [ ]:
def display_difference_example(authentic_dir, tampered_dir):
    # Pick a random tampered image
    tampered_files = [f for f in os.listdir(tampered_dir) if f.lower().endswith(('.jpg', '.tif'))]
    chosen_file = np.random.choice(tampered_files)
    tampered_img = cv2.imread(os.path.join(tampered_dir, chosen_file), cv2.IMREAD_COLOR)
    tampered_img_resized = cv2.resize(tampered_img, (224, 224))

    # Find its corresponding authentic image
    parts = chosen_file.split("_")
    if len(parts) >= 7:
        source_id = parts[4][:3] + "_" + parts[4][3:]
        target_id = parts[5][:3] + "_" + parts[5][3:]
        for id_ in [source_id, target_id]:
            for ext in ['.jpg', '.bmp']:
                auth_filename = f"Au_{id_}{ext}"
                auth_filepath = os.path.join(authentic_dir, auth_filename)
                if os.path.exists(auth_filepath):
                    auth_img = cv2.imread(auth_filepath, cv2.IMREAD_COLOR)
                    auth_img_resized = cv2.resize(auth_img, (224, 224))

                    # Calculate difference image
                    difference_img = calculate_difference_image(tampered_img_resized, auth_img_resized)

                    # Display images
                    plt.figure(figsize=(12, 4))
                    plt.subplot(1, 3, 1)
                    plt.imshow(cv2.cvtColor(tampered_img_resized, cv2.COLOR_BGR2RGB))
                    plt.title('Tampered Image')
                    plt.axis('off')

                    plt.subplot(1, 3, 2)
                    plt.imshow(cv2.cvtColor(auth_img_resized, cv2.COLOR_BGR2RGB))
                    plt.title('Authentic Image')
                    plt.axis('off')

                    plt.subplot(1, 3, 3)
                    plt.imshow(cv2.cvtColor(difference_img, cv2.COLOR_BGR2RGB))
                    plt.title('Difference Image')
                    plt.axis('off')

                    plt.show()
                    return

    print("No matching example found or unable to load images.")

In [ ]:
display_difference_example('D:\SASTRA Files\Sem-6\MINI PROJECT\Digital Image Forgery Detection\CASIA2 Dataset\Au')